In [1]:
import pandas as pd
import pickle

In [2]:
teams = {'Atlanta Falcons':'atl','Buffalo Bills':'buf','Carolina Panthers':'car','Chicago Bears':'chi',
         'Cincinnati Bengals':'cin','Cleveland Browns':'cle','Indianapolis Colts':'clt',
         'Arizona Cardinals':'crd','Dallas Cowboys':'dal','Denver Broncos':'den','Detroit Lions':'det',
         'Green Bay Packers':'gnb','Houston Texans':'htx','Jacksonville Jaguars':'jax',
         'Kansas City Chiefs':'kan','Miami Dolphins':'mia','Minnesota Vikings':'min','New Orleans Saints':'nor',
         'New England Patriots':'nwe','New York Giants':'nyg','New York Jets':'nyj','Tennessee Titans':'oti',
         'Philadelphia Eagles':'phi','Pittsburgh Steelers':'pit','Oakland Raiders':'rai',
         'Las Vegas Raiders':'rai','St. Louis Rams':'ram','Los Angeles Rams':'ram','Baltimore Ravens':'rav',
         'San Diego Chargers':'sdg','Los Angeles Chargers':'sdg','Seattle Seahawks':'sea',
         'San Francisco 49ers':'sfo','Tampa Bay Buccaneers':'tam','Washington Redskins':'was'}
cols = ['H_Pts','V_Pts','H_Pts_Opp','V_Pts_Opp','H_Off_Pass','V_Off_Pass','H_Off_Rush','V_Off_Rush',
        'H_Def_Pass','V_Def_Pass','H_Def_Rush','V_Def_Rush','H_TD','V_TD','H_TD_on_Def','V_TD_on_Def','H_FG_Made',
        'V_FG_Made','H_FG_Att','V_FG_Att','H_RZ_Conv','V_RZ_Conv','H_RZ_Att','V_RZ_Att','H_RZ_Def_Conv',
        'V_RZ_Def_Conv','H_RZ_Def_Att','V_RZ_Def_Att','H_Poss','V_Poss','H_Plays','V_Plays','H_TO_Gain','V_TO_Gain',
        'H_TO_Lost','V_TO_Lost','H_Yds_Pen','V_Yds_Pen','H_Sacks_Def','V_Sacks_Def','H_Tackles_Loss','V_Tackles_Loss',
        'H_Kickret','V_Kickret','H_Puntret','V_Puntret']
hdictmap = {'Points':'H_Pts','Points_Opp':'H_Pts_Opp','Yds_Off_Pass':'H_Off_Pass','Yds_Off_Rush':'H_Off_Rush',
           'Yds_Def_Pass':'H_Def_Pass','Yds_Def_Rush':'H_Def_Rush','TD':'H_TD','TD_on_Def':'H_TD_on_Def',
            'FG_Made':'H_FG_Made','FG_Att':'H_FG_Att','RZ_Conv':'H_RZ_Conv','RZ_Att':'H_RZ_Att',
            'RZ_Def_Conv':'H_RZ_Def_Conv','RZ_Def_Att':'H_RZ_Def_Att','Possession':'H_Poss','Plays':'H_Plays',
           'TO_Gained':'H_TO_Gain','TO_Lost':'H_TO_Lost','Yds_Pen':'H_Yds_Pen','Sacks_Def':'H_Sacks_Def',
           'Tackles_Loss':'H_Tackles_Loss','Yds_per_Kickret':'H_Kickret','Yds_per_Puntret':'H_Puntret'}
vdictmap = {'Points':'V_Pts','Points_Opp':'V_Pts_Opp','Yds_Off_Pass':'V_Off_Pass','Yds_Off_Rush':'V_Off_Rush',
           'Yds_Def_Pass':'V_Def_Pass','Yds_Def_Rush':'V_Def_Rush','TD':'V_TD','TD_on_Def':'V_TD_on_Def',
            'FG_Made':'V_FG_Made','FG_Att':'V_FG_Att','RZ_Conv':'V_RZ_Conv','RZ_Att':'V_RZ_Att',
            'RZ_Def_Conv':'V_RZ_Def_Conv','RZ_Def_Att':'V_RZ_Def_Att','Possession':'V_Poss','Plays':'V_Plays',
           'TO_Gained':'V_TO_Gain','TO_Lost':'V_TO_Lost','Yds_Pen':'V_Yds_Pen','Sacks_Def':'V_Sacks_Def',
           'Tackles_Loss':'V_Tackles_Loss','Yds_per_Kickret':'V_Kickret','Yds_per_Puntret':'V_Puntret'}

In [63]:
years = [str(year) for year in range(2010,2020)]

In [64]:
dfgame=pd.DataFrame(columns=cols)
dfgame.index.set_names('Code',inplace=True)

In [80]:
for year in years:
    path = 'data/df_step2_'+year+'.data'
    with open(path, 'rb') as f:
        dfyear = pickle.load(f)
    for team in dfyear.index.levels[0]: 
        dfyear.loc[team].sort_index(inplace=True)
        for row in range(5,len(dfyear.loc[team])):
            code = dfyear.loc[team].index[row]
            if team == code[-3:]:
                for key in hdictmap:
                    dfgame.loc[code,hdictmap[key]] = \
                        dfyear.loc[team].iloc[row-5:row,dfyear.columns.get_loc(key)].mean() # Averaged over 5 games
            else: 
                for key in vdictmap:
                    dfgame.loc[code,vdictmap[key]] = \
                        dfyear.loc[team].iloc[row-5:row,dfyear.columns.get_loc(key)].mean()

dfgame.dropna(inplace=True)
dfgame['H_TD_on_Def'] = 5.0 * dfgame['H_TD_on_Def'] # Use total rather than average
dfgame['V_TD_on_Def'] = 5.0 * dfgame['V_TD_on_Def'] # Use total rather than average
dfgame[cols] = dfgame[cols].astype('float64')

In [100]:
dfgame.describe().iloc[:,40:]

,H_Tackles_Loss,V_Tackles_Loss,H_Kickret,V_Kickret,H_Puntret,V_Puntret
count,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000
mean,4.675663,4.703114,20.125223,20.241859,7.668427,7.711081
std,1.076785,1.087208,5.373122,5.453188,4.085479,4.049717
min,1.400000,1.400000,3.150000,1.000000,-1.800000,-0.666667
25%,3.800000,4.000000,16.650000,16.800000,4.900000,4.900000
50%,4.600000,4.600000,20.166667,20.150000,6.963333,7.100000
75%,5.400000,5.400000,23.343333,23.498333,9.600000,9.765000
max,8.000000,9.200000,40.800000,42.140000,30.371429,31.104762


In [93]:
dfgame[dfgame['H_Pts'] == dfgame['H_Pts'].max()]

,H_Pts,V_Pts,H_Pts_Opp,V_Pts_Opp,H_Off_Pass,V_Off_Pass,H_Off_Rush,V_Off_Rush,H_Def_Pass,V_Def_Pass,...,H_Yds_Pen,V_Yds_Pen,H_Sacks_Def,V_Sacks_Def,H_Tackles_Loss,V_Tackles_Loss,H_Kickret,V_Kickret,H_Puntret,V_Puntret
Code,,,,,,,,,,,,,,,,,,,,,
201310130den,46.0,10.2,27.8,32.6,378.8,216.4,116.0,58.4,368.6,229.2,...,62.6,51.0,3.0,2.0,5.2,5.0,18.02,20.1,12.35,5.235714


In [70]:
dfgame

,H_Pts,V_Pts,H_Pts_Opp,V_Pts_Opp,H_Off_Pass,V_Off_Pass,H_Off_Rush,V_Off_Rush,H_Def_Pass,V_Def_Pass,...,H_Yds_Pen,V_Yds_Pen,H_Sacks_Def,V_Sacks_Def,H_Tackles_Loss,V_Tackles_Loss,H_Kickret,V_Kickret,H_Puntret,V_Puntret
Code,,,,,,,,,,,,,,,,,,,,,
201010170phi,24.4,22.6,20.6,14.0,255.4,233.0,130.4,148.8,212.4,239.8,...,69.8,51.8,3.0,2.2,5.0,4.4,19.800000,21.300000,13.446667,7.000000
201010240atl,24.2,20.0,17.2,20.4,232.6,257.4,150.2,103.4,257.8,213.6,...,53.0,54.8,1.8,1.2,5.0,2.6,21.866667,25.913333,8.600000,2.433333
201011070atl,23.8,19.8,22.2,28.4,247.4,252.2,136.6,104.0,307.4,217.8,...,46.2,54.4,2.0,0.4,5.0,2.4,19.923810,22.163333,6.200000,5.266667
201011110atl,23.8,26.2,21.6,19.6,248.8,253.2,122.2,136.6,272.2,272.0,...,48.8,48.6,1.8,1.8,5.0,4.2,24.663810,21.600000,6.200000,1.066667
201011210ram,16.6,25.8,22.4,23.0,196.2,257.4,114.4,114.6,249.8,277.6,...,64.0,37.8,3.8,2.0,6.0,4.4,36.000000,26.630476,8.666667,7.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912010rav,40.4,35.2,12.4,19.8,198.4,283.4,217.2,113.2,214.8,180.0,...,55.0,70.2,2.8,4.8,4.4,7.6,15.666667,12.733333,5.400000,4.633333
201911030sea,26.4,31.8,23.4,33.4,245.2,334.0,141.8,94.8,297.6,323.6,...,50.6,63.2,1.4,2.4,4.2,7.0,25.066667,9.100000,2.933333,4.293333
201911110sfo,27.8,29.0,9.6,28.2,213.4,272.8,168.8,147.8,134.8,316.4,...,38.0,46.6,4.2,1.0,6.6,3.2,9.400000,30.266667,13.766667,4.866667


In [ ]:
dfyear.index.levels[0]

In [85]:
path='data/df_analyze.pkl'
with open(path, 'wb') as f:
    pickle.dump(dfgame,f)

In [94]:
year='2013'
path='data/df_step2_'+year+'.data'
with open(path, 'rb') as f:
    dfyear = pickle.load(f)

In [95]:
dfyear.loc['den']

,Home/Away,Opponent,Points,Points_Opp,Yds_Off_Pass,Yds_Off_Rush,Yds_Def_Pass,Yds_Def_Rush,TD,TD_on_Def,...,RZ_Def_Att,Possession,Plays,TO_Gained,TO_Lost,Yds_Pen,Sacks_Def,Tackles_Loss,Yds_per_Kickret,Yds_per_Puntret
Code,,,,,,,,,,,,,,,,,,,,,
201309050den,H,Baltimore Ravens,49,27,462,65,362,58,7,0,...,6,26.200000,68,2,2,61,4.0,10,0.00,10.500000
201309150nyg,A,New York Giants,41,23,307,107,362,23,5,0,...,6,28.033333,74,4,1,132,1.0,7,17.00,30.250000
201309230den,H,Oakland Raiders,37,21,374,164,316,49,4,0,...,4,35.400000,76,0,2,40,3.0,2,22.50,0.000000
201309290den,H,Philadelphia Eagles,52,20,337,141,297,166,5,0,...,5,33.366667,72,0,0,25,3.0,3,26.00,21.000000
201310060dal,A,Dallas Cowboys,51,48,414,103,506,52,6,0,...,7,32.566667,76,2,2,55,4.0,4,24.60,0.000000
201310130den,H,Jacksonville Jaguars,35,19,295,112,303,71,6,1,...,4,28.750000,71,2,3,31,2.0,3,0.00,1.000000
201310200clt,A,Indianapolis Colts,33,39,386,64,228,121,4,0,...,7,28.216667,75,1,3,103,2.0,8,30.75,4.000000
201310270den,H,Washington Redskins,45,21,354,107,180,112,6,1,...,2,32.000000,81,5,4,57,3.0,3,0.00,18.000000
201311100sdg,A,San Diego Chargers,28,20,330,84,218,131,4,0,...,5,21.950000,60,0,1,28,4.0,7,25.75,3.000000
